In [1]:
import os, sys, email
import numpy as np
import pandas as pd
from nltk import tokenize
import pprint, re

from fileRetrieval import path_list
from fileToDataFrame import fileToDataFrame

In [2]:
# ディレクトリ 内のメールファイルを読み込む

directory_path = "enron/maildir/allen-p/"

file_list, dir_list=path_list(directory_path)

In [3]:
# テキストファイルをデータフレームに格納する

df = fileToDataFrame(file_list)

In [4]:
df.head()

,Path,Message-ID,Date,From,To,Cc,Bcc,X-From,X-To,X-cc,X-bcc,Subject,Message
0,enron/maildir/allen-p/notes_inbox/36.,<12357410.1075855679611.JavaMail.evans@thyme>,"Tue, 12 Dec 2000 04:41:00 -0800 (PST)",christi.nicolay@enron.com,phillip.allen@enron.com,None,None,Christi L Nicolay,Phillip K Allen,,,Re: Talking points about California Gas market,Phillip--To the extent that we can give Chair ...
1,enron/maildir/allen-p/notes_inbox/19.,<25849444.1075855679233.JavaMail.evans@thyme>,"Wed, 13 Dec 2000 08:11:00 -0800 (PST)",yild@zdemail.zdlists.com,pallen@enron.com,None,None,"""Y-Life to Go"" <yild@zdemail.zdlists.com>",pallen@ENRON.COM,,,Y-Life Daily: Bush will almost definitely be p...,"Y-Life Daily Bulletin: December 13, 2000\n\nNo..."
2,enron/maildir/allen-p/notes_inbox/50.,<18640335.1075855713056.JavaMail.evans@thyme>,"Thu, 10 May 2001 06:05:00 -0700 (PDT)",lisa.jacobson@enron.com,"lisa.jacobson@enron.com, kevin.mcgowan@enron.c...",None,None,Lisa Jacobson,"Lisa Jacobson, Kevin McGowan, Daniel Reck, Mat...",,,RSVP REQUESTED - Emissions Strategy Meeting....,"Due to some problems with my email yesterday, ..."
3,enron/maildir/allen-p/notes_inbox/3.,<32027475.1075855678667.JavaMail.evans@thyme>,"Wed, 13 Dec 2000 13:28:00 -0800 (PST)",subscriptions@intelligencepress.com,pallen@enron.com,None,None,subscriptions@intelligencepress.com,pallen@enron.com,,,"NGI Publications - Thursday, 14 December 2000","Dear phillip,\n\n\nThis e-mail is automated no..."
4,enron/maildir/allen-p/notes_inbox/9.,<14534717.1075855678819.JavaMail.evans@thyme>,"Wed, 13 Dec 2000 08:22:00 -0800 (PST)",rebecca.cantrell@enron.com,"stephanie.miller@enron.com, ruth.concannon@enr...",None,None,Rebecca W Cantrell,"Stephanie Miller, Ruth Concannon, Jane M Tholt...",,,Final Filed Version -- SDG&E Comments,FYI.\n---------------------- Forwarded by Rebe...


In [5]:
from toSentences import toSentences
cleaned_sentences = toSentences(df['Message'][0])

In [6]:
mail_df = pd.DataFrame({'Message-ID':df['Message-ID'][0],
                       'Sentence-ID':range(len(cleaned_sentences)),
                       'Sentence':cleaned_sentences})

In [7]:
triple_df = pd.DataFrame(columns=['Message-ID','Sentence-ID','Triple'])
triple_df

,Message-ID,Sentence-ID,Triple


In [8]:
from tripleExtraction import tripleExtraction

for i, sentence in enumerate(mail_df['Sentence']):
    triples = tripleExtraction(sentence)
    addrow = pd.DataFrame({'Message-ID':mail_df['Message-ID'][i],
                          'Sentence-ID':mail_df['Sentence-ID'][i],
                          'Triple':triples})
    triple_df = pd.concat([triple_df, addrow])

In [9]:
triple_df

,Message-ID,Sentence-ID,Triple
0,<12357410.1075855679611.JavaMail.evans@thyme>,0,"(we, like, to)"
0,<12357410.1075855679611.JavaMail.evans@thyme>,1,"(The Commission, getting, calls)"
1,<12357410.1075855679611.JavaMail.evans@thyme>,1,"(The Commission, provide, some response)"
0,<12357410.1075855679611.JavaMail.evans@thyme>,3,"(We, decided, what we will provide)"
1,<12357410.1075855679611.JavaMail.evans@thyme>,3,"(we, provide, what)"
0,<12357410.1075855679611.JavaMail.evans@thyme>,4,"(we, get, the numbers)"
0,<12357410.1075855679611.JavaMail.evans@thyme>,6,"(I, read, these points)"
1,<12357410.1075855679611.JavaMail.evans@thyme>,6,"(they, need, some touch up)"
0,<12357410.1075855679611.JavaMail.evans@thyme>,7,"(I, understand, why we need to give our commen..."
1,<12357410.1075855679611.JavaMail.evans@thyme>,7,"(we, need, to give our commentary on why price..."
